In [ ]:
from geopy import distance as geopy_dist
import os
from mapbox import Directions

def build_distance_matrix_mapbox(local_store_df):
    with open("./mapbox_api_key.txt") as f:
        api_key = f.readline()
    os.environ['MAPBOX_ACCESS_TOKEN'] = api_key
    service = Directions()
    
    n = len(local_store_df)
    distance_mat = [[0 for _ in range(n)] for _ in range(n)]
    
    for i in range(n):
        for j in range(n):
            if i == j: continue
            
            latlong_i = (local_store_df.iloc[i]["latitude"], local_store_df.iloc[i]["longitude"])
            latlong_j = (local_store_df.iloc[j]["latitude"], local_store_df.iloc[j]["longitude"])
            
            response = service.directions([latlong_i[::-1], latlong_j[::-1]], profile="mapbox/driving")
            
            if response.status_code != 200: raise Exception("Failed to retrieve routes from Mapbox API")
            
            response_json = response.json()
            
            if response_json["code"] == "NoRoute":
                print("No route found, use geodesic distance")
                distance_mat[i][j] = geopy_dist.geodesic(latlong_i, latlong_j).km
            else:
                distance_mat[i][j] = response_json["routes"][0]["distance"] / 1000
    
    return distance_mat

In [ ]:
from geopy import distance as geopy_dist

def build_distance_matrix(local_store_df):
    n = len(local_store_df)
    distance_mat = [[0 for _ in range(n)] for _ in range(n)]
    
    for i in range(n):
        for j in range(i+1):
            if i == j: continue
            
            latlong_i = (local_store_df.iloc[i]["latitude"], local_store_df.iloc[i]["longitude"])
            latlong_j = (local_store_df.iloc[j]["latitude"], local_store_df.iloc[j]["longitude"])
            
            geodesic_dist = geopy_dist.geodesic(latlong_i, latlong_j).km
            distance_mat[i][j] = distance_mat[j][i] = geodesic_dist
    
    return distance_mat

In [ ]:
import itertools
import sys


def held_karp(dists):
    """
    Implementation of Held-Karp, an algorithm that solves the Traveling
    Salesman Problem using dynamic programming with memoization.
    Parameters:
        dists: distance matrix
    Returns:
        A tuple, (cost, path).
    """
    n = len(dists)

    # Maps each subset of the nodes to the cost to reach that subset, as well
    # as what node it passed before reaching this subset.
    # Node subsets are represented as set bits.
    C = {}

    # Set transition cost from initial state
    for k in range(1, n):
        C[(1 << k, k)] = (dists[0][k], 0)
        # C[bin, k] = (dists[0][k], parent=0)
        print(f"C[{bin(1 << k)}=>{1 << k}, {k}] = {(dists[0][k], 0)}")
    
    # Iterate subsets of increasing length and store intermediate results
    # in classic dynamic programming manner
    powerset = list(subset for subset_size in range(2, n) for subset in itertools.combinations(range(1, n), subset_size))
    for subset in powerset:
        print(subset)
        # Set bits for all nodes in this subset
        bits = 0
        for bit in subset:
            # print(1 << bit, bin(1 << bit))
            bits |= 1 << bit
        print(bits, bin(bits))
        print()

        # Find the lowest cost to get to this subset
        for k in subset:
            prev = bits & ~(1 << k) 
            print(prev, bin(prev))
            res = []
            for m in subset:
                if m == 0 or m == k:
                    continue
                C[(prev, m)][0] + dists[m][k]
                res.append((C[(prev, m)][0] + dists[m][k], m))
            C[(bits, k)] = min(res)
            print("here")
            print(C)
    
    # We're interested in all bits but the least significant (the start state)
    bits = (2**n - 1) - 1

    # Calculate optimal cost
    res = []
    for k in range(1, n):
        res.append((C[(bits, k)][0] + dists[k][0], k))
    opt, parent = min(res)

    # Backtrack to find full path
    path = []
    for i in range(n - 1):
        path.append(parent)
        new_bits = bits & ~(1 << parent)
        _, parent = C[(bits, parent)]
        bits = new_bits

    # Add implicit start state
    path.append(0)

    return opt, list(reversed(path))

a, b = held_karp(distance_mat_nl)
# check later : https://leetcode.com/problems/parallel-courses-ii/discuss/817437/python3-a-la-bellman-held-karp